# Video understanding - shot 기반 챕터 분석
챕터 분석은 비디오를 구조화된 세그먼트로 나누고 각 세그먼트의 주요 오디오 및 시각적 콘텐츠를 요약합니다. 미디어 산업에서 이는 편집자, 프로듀서 및 분석가가 스토리라인을 빠르게 이해하고, 중요한 순간을 강조하며, TV 쇼, 영화, 다큐멘터리 및 뉴스 프로그램의 콘텐츠 인덱싱을 간소화하는 데 도움이 됩니다.

이 샘플 노트북에서는 Video Understanding tool을 사용하여 추출한 메타데이터를 사용합니다. 이 메타데이터에는 관련 timestamp가 포함된 shot 요약 및 오디오 transcript가 포함되어 있습니다. 이 메타데이터를 검색하여 LLM을 통한 추가 챕터 분석에 사용합니다.

![media_chapter](../statics/video-media-chapter.png)

이 분석에는 오픈 소스 영화인 [Meridian](https://en.wikipedia.org/wiki/Meridian_(film))을 사용합니다.

In [ ]:
from utils import dynamodb_tool, bedrock_tool
import json
import boto3
from IPython.display import JSON, Markdown, display

Video understanding tool의 결과는 Amazon S3와 Amazon DynamoDB 모두에 저장되며, 동일한 AWS 계정 내에서 직접 액세스할 수 있습니다.

이 lab에서는 `dynamodb_utils` 함수를 사용하여 DynamoDB에서 추출된 비디오 요약 및 오디오 transcription을 검색합니다. 이 tool은 4개의 DynamoDB 테이블을 관리합니다:

- Task 테이블: **bedrock_mm_extr_srv_video_task** — task ID, 상태, 생성 시간 및 원본 요청과 같은 비디오 처리 task와 관련된 메타데이터를 저장합니다.
- Frame 테이블: **bedrock_mm_extr_srv_video_frame** — task ID와 연결된 frame을 저장하며, frame 기반 pipeline에서 생성된 frame 수준 분석 결과를 포함합니다.
- Shot 테이블: **bedrock_mm_extr_srv_video_shot** — task ID와 연결된 shot 수준 추출 결과를 저장하며, shot 기반 pipeline에서 채워집니다.
- Transcript 테이블: **bedrock_mm_extr_srv_video_transcript** — 각 task ID와 연결된 오디오 transcription을 저장합니다.

Video Understanding Tool UI에서 task ID를 찾을 수 있습니다.

Shot-based 섹션으로 이동하여 이전에 UI를 사용하여 처리한 Meridan 비디오를 선택합니다. Get Task ID 링크를 클릭한 다음 아래 값을 해당 값으로 바꿉니다.

![Find task Id](../statics/find-task-id.png)


In [ ]:
task_id = 'YOUR_TASK_ID_FROM_VIDEO_UNDERSTANDING_TOOL'

Video Understanding Tool의 관리형 데이터베이스에서 transcript를 검색합니다.

In [ ]:
transcripts = dynamodb_tool.get_transcripts(task_id)
print(json.dumps(transcripts,indent=2, ensure_ascii=False))

Video Understanding Tool의 관리형 데이터베이스에서 shot 요약을 검색합니다.

In [ ]:
shots = dynamodb_tool.get_shot_outputs(task_id=task_id, output_names=["Summarize shot"])
print(json.dumps(shots,indent=2, ensure_ascii=False))

## transcript와 shot 요약을 모두 사용하여 챕터를 직접 분석
짧은 비디오(예: 10분 미만)의 경우 오디오 transcript와 shot 요약을 LLM에 직접 전달하여 챕터를 분석할 수 있습니다.

이 예제에서는 균형 잡힌 추론 기능을 제공하는 **Nova Pro**를 사용합니다. 이 모델은 큰 입력 크기(긴 비디오의 경우)를 처리하고 shot 요약과 오디오 transcript 간의 timestamp 정렬과 같은 추론 작업을 수행하는 데 적합합니다.

In [ ]:
# 데모의 처리 시간을 줄이기 위해 처음 30개의 shot만 입력으로 사용합니다
shots_subset = shots[0:30] if len(shots) > 30 else shots

prompt = f'''
당신은 비디오를 챕터 기반 세그먼트로 요약하는 미디어 전문가입니다.
챕터는 관련 콘텐츠, 이벤트 또는 장면을 함께 그룹화하는 비디오의 일관된 세그먼트입니다. 각 챕터에는 정의된 시작 및 종료 시간이 있으며 오디오 및 시각적 정보를 모두 포함할 수 있어 비디오의 해당 부분에 대한 요약된 보기를 제공합니다.
두 가지 입력이 제공됩니다:
    - timestamp가 있는 Shot 수준 시각적 설명.
    - timestamp가 있는 오디오 transcription 텍스트.

당신의 작업은 다음과 같습니다:
    - 시각적 정보와 오디오 정보를 병합하여 의미 있는 챕터를 식별합니다.
    - 인접한 frame과 transcript 세그먼트를 일관된 챕터로 그룹화합니다.

각 챕터에 대해 다음을 제공합니다:
    - 시작 시간(shot start_time/transcript의 가장 빠른 timestamp).
    - 종료 시간(shot end_time/transcript의 가장 늦은 timestamp).
    - 챕터 요약(시각적 및 오디오 컨텍스트를 결합한 간결한 설명).
    - 챕터 경계가 시각적, 주제 또는 대화의 주요 변화를 반영하도록 합니다.

결과를 markdown 형식으로 출력합니다.

Shot 요약:
{shots_subset}
오디오 Transcript:
{transcripts}
'''

In [ ]:
response = bedrock_tool.bedrock_converse(
    model_id="global.amazon.nova-2-lite-v1:0", 
    prompt=prompt, 
    inference_config={
      "maxTokens": 10000,
      "topP": 0.1,
      "temperature": 0.1
    }
)
result = bedrock_tool.parse_converse_response(response)
display(Markdown(result.replace("\\n", "\n")))

## 장편 비디오를 위한 챕터 분석
밀도 높은 shot 전환과 광범위한 transcript가 있는 2시간짜리 영화와 같은 긴 미디어의 경우, 이 모든 정보를 제공하면 LLM의 context window를 초과할 수 있으며, 더 큰 입력은 종종 덜 정확한 결과로 이어집니다. 좋은 최적화는 먼저 오디오 transcript를 챕터로 요약한 다음 이러한 챕터 경계를 사용하여 최종 챕터 수준 요약을 위해 시각적(shot 요약)을 그룹화하는 것입니다.

오디오 기반 챕터와 비전 기반 챕터 중 어느 것을 선택할지는 비디오 콘텐츠에 따라 다릅니다. TV 쇼, 영화, 다큐멘터리, 스포츠 또는 뉴스와 같이 전문적으로 편집되고 오디오가 많은 정보를 전달하는 대부분의 에피소드 비디오의 경우, 오디오 기반 챕터가 일반적으로 기준선으로 더 적합하며 추가 컨텍스트를 위해 시각적 정보가 혼합됩니다.

### 오디오 transcript를 챕터로 요약

In [ ]:
prompt = f'''
당신은 비디오 오디오 transcript를 일관되고 겹치지 않는 챕터로 요약하는 미디어 전문가입니다. 당신의 작업은 transcript를 분석하고 명확한 제목, 요약 및 시간 범위가 있는 챕터로 비디오를 세그먼트화하는 것입니다.
가이드라인:
- 챕터는 간격 없이 비디오 duration을 완전히 커버해야 합니다. 첫 번째 챕터는 0초부터 시작해야 합니다.
- 오디오 transcript를 사용할 수 없는 시간 범위의 경우 해당 duration에 걸쳐 기본 챕터(예: "오디오 Transcript 없음")를 생성합니다.
- 챕터는 순차적이고 연속적이어야 하며, 한 챕터의 종료 시간이 다음 챕터의 시작 시간과 일치해야 합니다.
- "오디오 Transcript 없음"이 연속될 경우 하나로 합칩니다.
오디오 Transcript:
{transcripts}
'''

Tool 구성은 Foundation Model에서 구조화된 출력을 얻는 더 안정적인 방법을 제공합니다. 이 예제에서는 다음 tool 구성을 정의하고 Bedrock Converse API로 전송합니다.

In [ ]:
tool_config = {
    "toolChoice": {
        "tool": {
            "name": "audio_chapters"
        }
    },
    "tools": [
        {
            "toolSpec": {
                "name": "audio_chapters",
                "description": "입력 오디오 transcript를 분석하고 명확한 제목, 요약 및 시간 범위가 있는 챕터로 비디오를 세그먼트화합니다.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "chapters": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "title": {
                                            "type": "string"
                                        },
                                        "start_time": {
                                            "type": "number"
                                        },
                                        "end_time": {
                                            "type": "number"
                                        },
                                        "audio_summary": {
                                            "type": "string"
                                        }
                                    },
                                    "required": [
                                        "title",
                                        "start_time",
                                        "end_time",
                                        "audio_summary"
                                    ]
                                }
                            }
                        },
                        "required": [
                            "chapters"
                        ]
                    }
                }
            }
        }
    ]
}

In [ ]:
response = bedrock_tool.bedrock_converse(
    model_id="global.amazon.nova-2-lite-v1:0", 
    prompt=prompt, 
    tool_config=tool_config,
    inference_config={
      "maxTokens": 10000,
      "temperature": 0.1,
      "topP" : 0.1
    }
)

result = bedrock_tool.parse_converse_response(response)
print(json.dumps(json.loads(result),indent=2,ensure_ascii=False))

### 오디오 챕터에 shot 정렬
오디오 transcript와 시각적(shot) 요약을 모두 사용하여 챕터 요약을 생성합니다.

In [ ]:
chapters = json.loads(result)
for chapter in chapters.get("chapters"):
    print(f'챕터 처리 중: {chapter["title"]} [{chapter.get("start_time")}-{chapter.get("end_time")}] 초')

    # 겹치는 shot 찾기
    overlapping_shots = [
        item for item in shots
        if item["start_time"] < chapter.get("end_time") and item["end_time"] > chapter.get("start_time")
    ]
    overlapping_shots_sorted = sorted(overlapping_shots, key=lambda x: x["start_time"])

    # 오디오 transcript 요약과 shot 요약을 사용하여 챕터 요약 생성
    chatper_prompt = f'''
    당신은 비디오에 대한 포괄적인 챕터 요약을 작성하는 임무를 맡은 미디어 전문가입니다. 당신의 작업은 오디오 transcript(챕터 요약)와 시각적 콘텐츠(shot 요약)의 정보를 결합하여 전체 비디오의 주요 이벤트와 세부 정보를 반영하는 통합되고 일관된 챕터 요약을 생성하는 것입니다.
    결과에 Markdown 접두사를 포함하지 마십시오. '비디오가 시작됩니다', '비디오가 열립니다'와 같은 선행 구문으로 요약을 시작하지 말고 비디오를 직접  설명하십시오.
    오디오 transcript 요약:
    {chapter.get("audio_summary")}
    요약이 있는 Vision shot:
    {overlapping_shots_sorted}
    '''
    response = bedrock_tool.bedrock_converse(
        model_id="global.amazon.nova-2-lite-v1:0", 
        prompt=chatper_prompt, 
        inference_config={
          "maxTokens": 10000,
          "temperature": 0.1,
          "topP": 0.1
        }
    )
    full_summary = bedrock_tool.parse_converse_response(response)
    chapter["full_summary"] = full_summary

In [ ]:
print(json.dumps(chapters,indent=2, ensure_ascii=False))

## 요약
비디오 챕터 분석은 복잡한 주제이며, 방법과 논리는 비디오 콘텐츠 유형과 비즈니스 요구 사항에 따라 달라질 수 있습니다. 이 샘플은 video understanding tool을 사용하여 추출한 메타데이터를 기반으로 비디오를 분석하는 두 가지 대표적인 접근 방식을 보여주며 시작점 역할을 합니다. 분석 흐름을 더욱 확장하고 사용자 정의하여 특정 요구 사항에 맞게 결과를 조정할 수 있습니다.